# Import everything

In [1]:
import cv2
from pprint import pprint
from requests import post


from ishelly.components.switch import SwitchSetRequest, SwitchSetParams
from ishelly.client import ShellyPlug


# Initialize plug object

In [2]:
plug_1 = ShellyPlug("http://192.168.1.201")


Getting device info
Base init completed


# Get the current config for reference

In [3]:
switch_config = plug_1.switch.get_config()
print("Got Config")
pprint(switch_config)

{'id': 1, 'src': 'shellyplugus-083af2004684', 'result': {'id': 0, 'name': 'electric burner #1', 'initial_state': 'on', 'auto_on': False, 'auto_on_delay': 60.0, 'auto_off': True, 'auto_off_delay': 5.0, 'power_limit': 4480, 'voltage_limit': 280, 'autorecover_voltage_errors': False, 'current_limit': 16.0}}
Got Config
SwitchConfig(id=0, name='electric burner #1', initial_state='on', auto_on=False, auto_on_delay=60, auto_off=True, auto_off_delay=5, autorecover_voltage_errors=False, input_id=None, power_limit=4480, voltage_limit=280, undervoltage_limit=None, current_limit=16, restart_required=None)


In [4]:
switch_status = plug_1.switch.get_status()
print("Got Status")
print(switch_status)

Got Status
id=0 source='timer' output=False timer_started_at=None timer_duration=None apower=0.0 voltage=124.1 current=0.0 pf=None freq=None aenergy=ActiveEnergy(total=5847.442, by_minute=[0.0, 0.0, 0.0], minute_ts=1706199253) temperature=Temperature(tC=46.8, tF=116.3) errors=None


# List Current Scheduled Actions

In [5]:
current_schedule = plug_1.schedule.list()
for task in current_schedule.jobs:
    print(task)

id=2 enable=False timespec='*/30 * * * * *' calls=[JSONRPCRequest(jsonrpc='2.0', id=1, method='Switch.Set', params={'id': 0, 'on': True, 'toggle_after': None})]
id=3 enable=True timespec='0 0 7 * * *' calls=[JSONRPCRequest(jsonrpc='2.0', id=1, method='Switch.Set', params={'id': 0, 'on': True, 'toggle_after': 0})]


# Create PID Control Schedule

Dont run this if the list schedule shows this schedule already exists. Instead we can enable/disable the scheduled task.

In [ ]:
timespec1 = "*/30 * * * * *"

job = SwitchSetRequest(
    id=1,
    params=SwitchSetParams(id=0, on=True),
)
plug_1.schedule.create(enable=True, timespec=timespec1, calls=[job])

# Enable/Disable PID Control Schedule

Enable

In [6]:
pid_control_active = True

plug_1.schedule.update(2, enabled=pid_control_active)
print(f"PID Schedule Active: {plug_1.schedule.list().jobs[0].enable}")


PID Schedule Active: True


Disable

In [16]:
pid_control_active = False

plug_1.schedule.update(2, enabled=pid_control_active)
print(f"PID Schedule Active: {plug_1.schedule.list().jobs[0].enable}")

PID Schedule Active: False


# Create heater control

In [7]:
loop_time_interval = 30

def set_heater_power(level):
    # level = 0-100
    print(f"using power level: {level}")
    old_config = plug_1.switch.get_config()
    new_config = old_config.model_copy()
    new_config.initial_state = "on"
    new_auto_off_interval = max(1, int(level * 0.01 * loop_time_interval))
    time_off = loop_time_interval - new_auto_off_interval
    print(f"new on/off interval seconds: {new_auto_off_interval} (time on) / {time_off} (time off)")
    new_config.auto_off_delay = new_auto_off_interval
    plug_1.switch.set_config(new_config)

In [8]:
set_heater_power(100)

using power level: 100
{'id': 1, 'src': 'shellyplugus-083af2004684', 'result': {'id': 0, 'name': 'electric burner #1', 'initial_state': 'on', 'auto_on': False, 'auto_on_delay': 60.0, 'auto_off': True, 'auto_off_delay': 5.0, 'power_limit': 4480, 'voltage_limit': 280, 'autorecover_voltage_errors': False, 'current_limit': 16.0}}
new on/off interval seconds: 30 (time on) / 0 (time off)
{'id': 1, 'src': 'shellyplugus-083af2004684', 'result': {'restart_required': False}}
